In [1]:
import math, time
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("taetfp.csv", encoding='big5')

In [3]:
df["代碼"]='00'+df["代碼"].apply(str)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19665 entries, 0 to 19664
Data columns (total 8 columns):
代碼         19665 non-null object
日期         19665 non-null int64
中文簡稱       19665 non-null object
開盤價(元)     19665 non-null float64
最高價(元)     19665 non-null float64
最低價(元)     19665 non-null float64
收盤價(元)     19665 non-null float64
成交張數(張)    19665 non-null object
dtypes: float64(4), int64(1), object(3)
memory usage: 1.2+ MB


In [5]:
df.shape

(19665, 8)

# Using fb prophet

In [6]:
df_submit2 = pd.DataFrame(columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])

In [7]:
df_submit2

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice


In [8]:
symbols = list(set(df["代碼"]))

In [9]:
from fbprophet import Prophet

In [11]:
for i in range(0, 18):
    df4 = pd.DataFrame() 
    symbol2 = symbols[i]
    df3 = df[df["代碼"] == symbols[i]].reset_index()
    df4['ds']=df3["日期"].apply(str).str[0:4]+'-'+df3["日期"].apply(str).str[4:6]+'-'+df3["日期"].apply(str).str[6:8]
    df4['y'] = np.log(df3['收盤價(元)'])            
    m = Prophet(interval_width=0.95) # 調整基本設定值
    m.fit(df4)    
    future = m.make_future_dataframe(periods=5)    
    forecast = m.predict(future)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']]=np.exp(forecast[['yhat', 'yhat_lower', 'yhat_upper']])
    df4['y']=np.exp(df4['y'])
    forecast=pd.concat([forecast, df4['y']], axis=1)
    forecast['yhat2']=forecast['yhat'].shift(1)
    forecast['y2']=forecast['y'].shift(1)
    ud1=1
    ud2=1
    ud3=1
    ud4=1
    ud5=1
    if forecast.iloc[len(forecast)-5]['yhat'] - forecast.iloc[len(forecast)-5]['y2'] <0: ud1= -1
    if forecast.iloc[len(forecast)-4]['yhat'] - forecast.iloc[len(forecast)-4]['yhat2'] <0 : ud2= -1
    if forecast.iloc[len(forecast)-3]['yhat'] - forecast.iloc[len(forecast)-3]['yhat2'] <0 : ud3= -1
    if forecast.iloc[len(forecast)-2]['yhat'] - forecast.iloc[len(forecast)-2]['yhat2'] <0 : ud4= -1
    if forecast.iloc[len(forecast)-1]['yhat'] - forecast.iloc[len(forecast)-1]['yhat2'] <0 : ud5= -1
    
    df_temp2 =  pd.DataFrame([[symbol2,ud1,forecast.iloc[len(forecast)-5]['yhat'],
                                      ud2,forecast.iloc[len(forecast)-4]['yhat'],
                                      ud3,forecast.iloc[len(forecast)-3]['yhat'],
                                      ud4,forecast.iloc[len(forecast)-2]['yhat'],
                                      ud5,forecast.iloc[len(forecast)-1]['yhat']]],
                             columns=['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice','Wed_ud','Wed_cprice',
                                  'Thu_ud','Thu_cprice','Fri_ud','Fri_cprice'])
    df_submit2=df_submit2.append(df_temp2)

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=Tr

In [12]:
df_submit2=df_submit2.sort_values(by=['ETFid'])

In [13]:
df_submit2.to_csv('ETF_predict_fb.csv', index=False)